In [ ]:
!pip install "protobuf<5" chromadb sentence-transformers PyPDF2


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import uuid
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb
import shutil
from tqdm import tqdm

# Define paths
pdf_folder_path = "AAAI_Papers"
chroma_db_path = "chroma_db"
export_path = "Chroma_db"

# Initialize ChromaDB with persistence
client = chromadb.PersistentClient(path=chroma_db_path)

# Load domain-specific embedding model
model = SentenceTransformer("allenai-specter")  # Fine-tuned for academic texts

# Create ChromaDB collection
collection = client.get_or_create_collection(
    name="research_papers",
    metadata={"distance_metric": "cosine"}
)

# Generate a unique identifier for this run
run_id = str(uuid.uuid4())[:8]

# Optional: Clear existing data in the collection
clear_collection = True
if clear_collection:
    collection.delete(where={"*": "*"})
    print("Cleared existing collection data.")

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {e}")
        return ""

# Function to chunk text with overlap
def chunk_text_with_overlap(text, chunk_size, overlap):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# Chunking configuration
chunk_size = 500
overlap_size = 100  # Added overlap for better context

# Process PDFs and add to ChromaDB
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

for doc_id, pdf_file in enumerate(tqdm(pdf_files, desc="Processing PDFs")):
    pdf_path = os.path.join(pdf_folder_path, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    if not text.strip():  # Skip empty or unreadable PDFs
        continue

    chunks = chunk_text_with_overlap(text, chunk_size, overlap_size)

    for chunk_id, chunk in enumerate(chunks):
        try:
            embedding = model.encode(chunk).tolist()
            collection.add(
                ids=[f"{run_id}_{doc_id}_{chunk_id}"],  # Unique ID with run identifier
                documents=[chunk],
                metadatas=[{
                    "document_id": doc_id,
                    "chunk_id": chunk_id,
                    "file_name": pdf_file
                }],
                embeddings=[embedding],
            )
        except Exception as e:
            print(f"Error processing chunk {chunk_id} of {pdf_file}: {e}")

# Check if data was added successfully
print(f"Number of documents in collection: {collection.count()}")

# Save ChromaDB to Google Drive incrementally
if os.path.exists(chroma_db_path):
    shutil.copytree(chroma_db_path, export_path, dirs_exist_ok=True)
    print(f"ChromaDB exported to {export_path}")
else:
    print("ChromaDB directory not found!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

...

Error processing chunk 39 of 31262.pdf: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]


Processing PDFs:  46%|████▌     | 1126/2440 [1:09:56<1:20:33,  3.68s/it]

Error processing chunk 77 of 28619.pdf: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Error processing chunk 78 of 28619.pdf: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Error processing chunk 79 of 28619.pdf: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]


Processing PDFs: 100%|██████████| 2440/2440 [2:28:43<00:00,  3.66s/it]


Number of documents in collection: 268809
ChromaDB exported to /content/drive/MyDrive/Chroma_db
